# Behavioural cloning project

In [1]:
# Configure matlab to show graphics in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Change to 'data' when training on a GPU
PATH_TO_DATA = 'all_data'

In [3]:
# Shape of the image
input_shape = (66, 200, 3)

In [4]:
def img_folder(data_folder):
    return '{}/IMG'.format(data_folder)

def path_driving_log(data_folder):
    return '{}/driving_log.csv'.format(data_folder)

In [5]:
datasets = ['train', 'test', 'valid']

In [6]:
def get_driving_log_dataframe(data_folder):
    driving_log_df = pd.read_csv(path_driving_log(data_folder))
    return driving_log_df

In [7]:
driving_log_df = get_driving_log_dataframe(PATH_TO_DATA)

In [8]:
zero_approximation = 10e-4

In [9]:
num_positive_steerings = len(driving_log_df[driving_log_df['steering'] > zero_approximation])
num_negative_steerings = len(driving_log_df[driving_log_df['steering'] < -zero_approximation])
num_neutral_steerings = len(driving_log_df[abs(driving_log_df['steering']) <= zero_approximation])

print('Positive: {}'.format(num_positive_steerings))
print('Negative: {}'.format(num_negative_steerings))
print('Neutral: {}'.format(num_neutral_steerings))

Positive: 2645
Negative: 3477
Neutral: 8694


In [10]:
positive_steering_weight = 10
negative_steering_weight = num_positive_steerings * 10/ num_negative_steerings
neutral_steering_weight = num_positive_steerings * 10 / num_neutral_steerings

In [11]:
path_to_folders = dict(zip(datasets, map(lambda folder: '{0}/{1}'.format(PATH_TO_DATA, folder), datasets)))

In [12]:
path_to_folders

{'test': 'all_data/test', 'train': 'all_data/train', 'valid': 'all_data/valid'}

In [13]:
from scipy.ndimage import imread
from os import listdir
from sklearn.utils import shuffle

from PIL import Image

import math

def image_label_generator(data_folder, batch_size=64):
    driving_log_df = get_driving_log_dataframe(data_folder)
    number_of_examples = len(driving_log_df)
    image_columns = ['center', 'left', 'right']
    
    X_train = []
    y_train = []
    weights = []
    index_in_batch = 0
    batch_number = 0
    
    while True:
        for image_column in image_columns:
            image_series = driving_log_df[image_column]
            steering_series = driving_log_df['steering']
            for offset in range(0, number_of_examples, batch_size):
                X_train = []
                y_train = []
                weights = []

                end_of_batch = min(number_of_examples, offset + batch_size)

                for j in range(offset, end_of_batch):
                    image_filename = image_series[j].lstrip().rstrip()
                    image = Image.open('{0}/{1}'.format(data_folder, image_filename))
                    image = np.asarray(image.resize((200, 66)))
                    label = steering_series[j]
                    
#                     weights.append(1)
                    if abs(label) < 10e-4:
                        weights.append(neutral_steering_weight + math.exp(abs(label)))
                    elif label > 0:
                            weights.append(positive_steering_weight + math.exp(abs(label)))
                    else:
                        weights.append(negative_steering_weight + math.exp(abs(label)))
                    
                    
                    X_train.append(image)
                    y_train.append(label)
                    X_train, y_train, weights = shuffle(X_train, y_train, weights)

                yield np.array(X_train), np.array(y_train), np.array(weights)

In [14]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import Flatten
from keras.layers import Dropout

Using TensorFlow backend.


In [15]:
model = Sequential()

In [16]:
model.add(Convolution2D(3, 1, 1, 
                          input_shape=input_shape, 
                          border_mode='same', 
                          activation='relu',
                          init='he_normal'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Convolution2D(24, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Convolution2D(36, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Convolution2D(48, 
                        5, 5,
                       subsample=(2, 2),
                       init='he_normal'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Convolution2D(64,
                       3, 3,
                       init='he_normal'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Convolution2D(64,
                       3, 3,
                       init='he_normal'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(100, activation='relu', init='he_normal'))
model.add(Dense(50, activation='relu', init='he_normal'))
model.add(Dense(10, activation='relu', init='he_normal'))
model.add(Dense(1, activation='linear', init='he_normal'))

In [17]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 66, 200, 3)    12          convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 66, 200, 3)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 66, 200, 3)    12          dropout_1[0][0]                  
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 31, 98, 24)    1824        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [18]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath)
callbacks_list = [checkpoint]

In [19]:
from keras.optimizers import Adam
adam = Adam(lr=10e-4)

In [20]:
model.compile(optimizer=adam, 
              loss='mse',
             metrics=['mean_squared_error'])

In [21]:
image_generator = image_label_generator(PATH_TO_DATA)
samples = len(get_driving_log_dataframe(PATH_TO_DATA))

In [22]:
image_generator = image_label_generator(PATH_TO_DATA)
samples = len(get_driving_log_dataframe(PATH_TO_DATA))
model.fit_generator(image_generator, 
                    samples_per_epoch=samples, 
                    nb_epoch=25,
                   callbacks=callbacks_list)

Epoch 1/25
14816/14816 [==============================] - 52s - loss: 0.3442 - mean_squared_error: 0.0379    
Epoch 2/25
14816/14816 [==============================] - 48s - loss: 0.2844 - mean_squared_error: 0.0300    
Epoch 3/25
14816/14816 [==============================] - 48s - loss: 0.2758 - mean_squared_error: 0.0297    
Epoch 4/25
14816/14816 [==============================] - 48s - loss: 0.2869 - mean_squared_error: 0.0298    
Epoch 5/25
14816/14816 [==============================] - 48s - loss: 0.2592 - mean_squared_error: 0.0273    
Epoch 6/25
14816/14816 [==============================] - 48s - loss: 0.2214 - mean_squared_error: 0.0250    
Epoch 7/25
14816/14816 [==============================] - 48s - loss: 0.2030 - mean_squared_error: 0.0231    
Epoch 8/25
14816/14816 [==============================] - 48s - loss: 0.1905 - mean_squared_error: 0.0221    
Epoch 9/25
14816/14816 [==============================] - 48s - loss: 0.1854 - mean_squared_error: 0.0217    
Epoch 10/2

In [23]:
X_val, y_val, w = next(image_label_generator(PATH_TO_DATA))

In [24]:
model.predict(X_val)

array([[ 0.24417001],
       [-0.03308296],
       [-0.05282918],
       [-0.02040869],
       [-0.07171255],
       [-0.01176016],
       [-0.06641157],
       [-0.01749383],
       [-0.05876631],
       [-0.02088972],
       [ 0.22061512],
       [-0.01328376],
       [-0.01015107],
       [-0.02610557],
       [-0.0138338 ],
       [-0.02636448],
       [-0.03744147],
       [-0.00980639],
       [-0.03705426],
       [-0.01788818],
       [-0.02097031],
       [-0.01415297],
       [-0.00916732],
       [-0.01469635],
       [-0.03988317],
       [-0.01290986],
       [-0.02484368],
       [-0.02693775],
       [-0.00648441],
       [-0.00314726],
       [-0.01802123],
       [-0.06115625],
       [ 0.17940703],
       [-0.04470982],
       [-0.00487347],
       [-0.03361421],
       [-0.06399942],
       [-0.00702863],
       [ 0.20953931],
       [-0.02667213],
       [-0.00509981],
       [-0.0385359 ],
       [-0.0129332 ],
       [-0.02023245],
       [-0.02734696],
       [-0

In [25]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [26]:
y_val

array([ 0.3127406 , -0.05468059,  0.        , -0.2021756 ,  0.        ,
        0.        ,  0.        , -0.07803941, -0.3285842 , -0.2567704 ,
        0.        ,  0.        , -0.3898351 , -0.4916646 , -0.164727  ,
        0.        ,  0.        , -0.2008467 , -0.05077529,  0.        ,
       -0.512769  ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.3087602 ,  0.        , -0.4662809 , -0.2275407 ,
        0.        ,  0.        , -0.3237014 ,  0.        , -0.1233637 ,
        0.        ,  0.        ,  0.        ,  0.        , -0.06370211,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.4099739 ,  0.        ,  0.        ,  0.        , -0.357723  ,
       -0.00134849,  0.        ,  0.        , -0.3571165 ,  0.        ,
        0.        ,  0.        ,  0.        , -0.3544564 , -0.1534824 ,
       -0.6370363 ,  0.        , -0.2147133 , -1.        ])

In [24]:
from keras.models import model_from_json

with open('model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [26]:
loaded_model.compile(optimizer=adam, 
              loss='mse',
             metrics=['mean_squared_error'])

image_generator = image_label_generator(PATH_TO_DATA)
samples = len(get_driving_log_dataframe(PATH_TO_DATA))

loaded_model.fit_generator(image_generator, 
                    samples_per_epoch=samples, 
                    nb_epoch=10,
                   callbacks=callbacks_list)

Epoch 1/10
14816/14816 [==============================] - 50s - loss: 0.3507 - mean_squared_error: 0.0380    
Epoch 2/10
14816/14816 [==============================] - 48s - loss: 0.2957 - mean_squared_error: 0.0293    
Epoch 3/10
14816/14816 [==============================] - 48s - loss: 0.2632 - mean_squared_error: 0.0268    
Epoch 4/10
14816/14816 [==============================] - 48s - loss: 0.2357 - mean_squared_error: 0.0250    
Epoch 5/10
14816/14816 [==============================] - 48s - loss: 0.2257 - mean_squared_error: 0.0248    
Epoch 6/10
14816/14816 [==============================] - 48s - loss: 0.2068 - mean_squared_error: 0.0233    
Epoch 7/10
14816/14816 [==============================] - 48s - loss: 0.2041 - mean_squared_error: 0.0234    
Epoch 8/10
14816/14816 [==============================] - 48s - loss: 0.2040 - mean_squared_error: 0.0233    
Epoch 9/10
14816/14816 [==============================] - 48s - loss: 0.1937 - mean_squared_error: 0.0221    
Epoch 10/1